In [1]:
from Lib.myModules.my_ML_Modules_1 import *
import math
import sys
import contextlib

In [2]:
data_dir = "../data"
infantNum = 5
ECG_dataset = f"{data_dir}/infant{infantNum}_ecg"
RESP_dataset = f"{data_dir}/infant{infantNum}_resp"
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_ecg")
signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_resp")
print(f'ECG DATA: {signal_ecg_0[1]}')
print(F'RESP DATA: {signal_resp_0[1]}')
totalDuration_ECG = num2Time(num=signal_ecg_0[1]['sig_len'], fs=signal_ecg_0[1]['fs'])
print(f'{totalDuration_ECG} sec, {secs2minutes(totalDuration_ECG)} minutes, {secs2hours(totalDuration_ECG)} hours')
totalDuration_RESP = num2Time(num=signal_resp_0[1]['sig_len'], fs=signal_resp_0[1]['fs'])
print(f'{totalDuration_RESP} sec, {secs2minutes(totalDuration_RESP)} minutes, {secs2hours(totalDuration_RESP)} hours')

ECG DATA: {'fs': 250, 'sig_len': 43878723, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['ECG'], 'comments': []}
RESP DATA: {'fs': 50, 'sig_len': 8955598, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['NU'], 'sig_name': ['RESP'], 'comments': []}
175514.892 sec, 2925.2482 minutes, 48.75413666666667 hours
179111.96 sec, 2985.199333333333 minutes, 49.75332222222222 hours


In [7]:
print(type(signal_ecg_0))

<class 'tuple'>


In [3]:
# initialize the training
start_time_secs = hours2secs(2)
end_time_secs = hours2secs(2.2)
lr_model, pr_model, svr_rbf_model = training_pipeline(start_time_secs=start_time_secs, 
                                                        end_time_secs=end_time_secs, 
                                                        signal_ecg_0=signal_ecg_0,
                                                        signal_resp_0=signal_resp_0,
                                                        ECG_dataset=ECG_dataset,
                                                        RESP_dataset=RESP_dataset)

start: 1800000, end: 1980000
start: 360000, end: 396000
90th percentile: 0.23355740272677641, 10th percentile: -0.3434667687158477, IQR: 0.5770241714426241
Identified outliers: 839
Data Shape: (180000,)
90th percentile: 1.1196942158927843, 10th percentile: 1.0967823235746934, IQR: 0.022911892318090876
Identified outliers: 410
Data Shape: (36000,)
[[149.31515351   8.15154173]
 [  8.15154173  28.63247294]]
Pearsons correlation: 0.125
Spearmans correlation: 0.105
---------------------------------------------
--- Linear Regression ---
Coefficient of determination: 0.01684401729962348
Intercept: [129.97673]
Coefficients: [[0.05660592]]
Mean absolute error: 2.67
Mean squared error: 29.11
Root mean squared error: 5.40
---------------------------------------------
--- Polynomial Regression ---
Coefficient of determination: 0.01219179173182594
Intercept: [130.67822]
Coefficients: [[ 2.2605619e-10  1.7824455e-09  2.7192581e-07  4.7672206e-06
  -1.4564542e-07  1.1390446e-09]]
Mean absolute error:

In [4]:
step = 0.25
startHrs = 10
endHrs = 12
logFilePath = f"../logs/infant{infantNum}_step{step}_startHrs{startHrs}_endHrs{endHrs}.log"

with open(logFilePath, 'a') as f:
    with contextlib.redirect_stdout(f):
        for i in np.arange(startHrs, endHrs, step):
            print("-------------------------------------------------------------------------")
            print(f"Data bin index: {i:.3f}")
            start_time_secs = hours2secs(i)
            end_time_secs = hours2secs(i+step)
            lr_model, pr_model, svr_rbf_model = cascade_training_pipeline(lr_model=lr_model, 
                                                                            pr_model=pr_model, 
                                                                            svr_rbf_model=svr_rbf_model, 
                                                                            start_time_secs=start_time_secs, 
                                                                            end_time_secs=end_time_secs,
                                                                            signal_ecg_0=signal_ecg_0,
                                                                            signal_resp_0=signal_resp_0,
                                                                            ECG_dataset=ECG_dataset,
                                                                            RESP_dataset=RESP_dataset)

In [5]:
# save the model
for i in [lr_model, pr_model, svr_rbf_model]:
    filename = f"../models/infant_{infantNum}/{namestr(i, globals())[0]}_step{step}_startHrs{startHrs}_endHrs{endHrs}.sav"
    joblib.dump(i, filename)
